# Getting Started with maude_db

**Purpose**: Complete introduction to local setup and helper methods  
**Data**: 2022-2023 (2 years), device + master tables  
**Runtime**: ~4 minutes

## What You'll Learn

- Database creation and persistence
- Multi-year downloads with master table
- Using `query_device()` helper method
- Event type breakdowns (deaths, injuries, malfunctions)
- Simple trend analysis
- Checksum tracking for data integrity


In [9]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent / 'src'))

from maude_db import MaudeDatabase
import pandas as pd

pd.set_option('display.max_columns', None)
print("✓ Setup complete!")

✓ Setup complete!


## 1. Create Database & Download Multi-Year Data

Download 2 years with both device and master tables. The master table contains event types and dates.

In [ ]:
db = MaudeDatabase('getting_started.db', verbose=True)

# Download 2022-2023 data
db.add_years(
    years='2022-2023',
    tables=['device', 'master', 'patient'],
    download=True,
    data_dir='./maude_data',
    interactive=False,
    force_refresh=True
)

print("\n✓ Data loaded!")
db.info()


Grouping years by file for optimization...

  Using cached device2022.zip
  Using cached device2023.zip
  Using mdrfoithru2024.zip instead (latest available cumulative file).
  Using cached mdrfoithru2024.zip
  Using patientthru2024.zip instead (latest available cumulative file).
  Using cached patientthru2024.zip

Processing data files...

Loading device for year 2022...
  File changed, refreshing years: [2022]
  Deleting old data for device year 2022...
    Identified date columns: DATE_REMOVED_FLAG, IMPLANT_DATE_YEAR, DATE_REMOVED_YEAR, DATE_RECEIVED, EXPIRATION_DATE_OF_DEVICE, DATE_RETURNED_TO_MANUFACTURER
    Processed 1,100,000 rows...
    Processed 2,100,000 rows...


## 2. Using query_device() Helper

This helper method joins device + master tables automatically.

In [ ]:
# Query pacemaker devices
results = db.query_device(device_name='pacemaker')

print(f"Found {len(results):,} pacemaker reports")
print(f"\nColumns available: {results.columns.tolist()[:10]}...")
results.head()

Found 129,615 pacemaker reports

Columns available: ['MDR_REPORT_KEY', 'EVENT_KEY', 'REPORT_NUMBER', 'REPORT_SOURCE_CODE', 'MANUFACTURER_LINK_FLAG_', 'NUMBER_DEVICES_IN_EVENT', 'NUMBER_PATIENTS_IN_EVENT', 'DATE_RECEIVED', 'ADVERSE_EVENT_FLAG', 'PRODUCT_PROBLEM_FLAG']...


,MDR_REPORT_KEY,EVENT_KEY,REPORT_NUMBER,REPORT_SOURCE_CODE,MANUFACTURER_LINK_FLAG_,NUMBER_DEVICES_IN_EVENT,NUMBER_PATIENTS_IN_EVENT,DATE_RECEIVED,ADVERSE_EVENT_FLAG,PRODUCT_PROBLEM_FLAG,DATE_REPORT,DATE_OF_EVENT,REPROCESSED_AND_REUSED_FLAG,REPORTER_OCCUPATION_CODE,HEALTH_PROFESSIONAL,INITIAL_REPORT_TO_FDA,DATE_FACILITY_AWARE,REPORT_DATE,REPORT_TO_FDA,DATE_REPORT_TO_FDA,EVENT_LOCATION,DATE_REPORT_TO_MANUFACTURER,MANUFACTURER_CONTACT_T_NAME,MANUFACTURER_CONTACT_F_NAME,MANUFACTURER_CONTACT_L_NAME,MANUFACTURER_CONTACT_STREET_1,MANUFACTURER_CONTACT_STREET_2,MANUFACTURER_CONTACT_CITY,MANUFACTURER_CONTACT_STATE,MANUFACTURER_CONTACT_ZIP_CODE,MANUFACTURER_CONTACT_ZIP_EXT,MANUFACTURER_CONTACT_COUNTRY,MANUFACTURER_CONTACT_POSTAL,MANUFACTURER_CONTACT_AREA_CODE,MANUFACTURER_CONTACT_EXCHANGE,MANUFACTURER_CONTACT_PHONE_NO,MANUFACTURER_CONTACT_EXTENSION,MANUFACTURER_CONTACT_PCOUNTRY,MANUFACTURER_CONTACT_PCITY,MANUFACTURER_CONTACT_PLOCAL,MANUFACTURER_G1_NAME,MANUFACTURER_G1_STREET_1,MANUFACTURER_G1_STREET_2,MANUFACTURER_G1_CITY,MANUFACTURER_G1_STATE_CODE,MANUFACTURER_G1_ZIP_CODE,MANUFACTURER_G1_ZIP_CODE_EXT,MANUFACTURER_G1_COUNTRY_CODE,MANUFACTURER_G1_POSTAL_CODE,DATE_MANUFACTURER_RECEIVED,MFR_REPORT_TYPE,DEVICE_DATE_OF_MANUFACTURE,SINGLE_USE_FLAG,REMEDIAL_ACTION,PREVIOUS_USE_CODE,REMOVAL_CORRECTION_NUMBER,EVENT_TYPE,DISTRIBUTOR_NAME,DISTRIBUTOR_ADDRESS_1,DISTRIBUTOR_ADDRESS_2,DISTRIBUTOR_CITY,DISTRIBUTOR_STATE_CODE,DISTRIBUTOR_ZIP_CODE,DISTRIBUTOR_ZIP_CODE_EXT,REPORT_TO_MANUFACTURER,MANUFACTURER_NAME,MANUFACTURER_ADDRESS_1,MANUFACTURER_ADDRESS_2,MANUFACTURER_CITY,MANUFACTURER_STATE_CODE,MANUFACTURER_ZIP_CODE,MANUFACTURER_ZIP_CODE_EXT,MANUFACTURER_COUNTRY_CODE,MANUFACTURER_POSTAL_CODE,TYPE_OF_REPORT,SOURCE_TYPE,DATE_ADDED,DATE_CHANGED,REPORTER_STATE_CODE,REPORTER_COUNTRY_CODE,PMA_PMN_NUM,EXEMPTION_NUMBER,SUMMARY_REPORT,NOE_SUMMARIZED,SUPPL_DATES_FDA_RECEIVED,SUPPL_DATES_MFR_RECEIVED,MDR_REPORT_KEY,DEVICE_EVENT_KEY,IMPLANT_FLAG,DATE_REMOVED_FLAG,DEVICE_SEQUENCE_NO,IMPLANT_DATE_YEAR,DATE_REMOVED_YEAR,SERVICED_BY_3RD_PARTY_FLAG,DATE_RECEIVED,BRAND_NAME,GENERIC_NAME,MANUFACTURER_D_NAME,MANUFACTURER_D_ADDRESS_1,MANUFACTURER_D_ADDRESS_2,MANUFACTURER_D_CITY,MANUFACTURER_D_STATE_CODE,MANUFACTURER_D_ZIP_CODE,MANUFACTURER_D_ZIP_CODE_EXT,MANUFACTURER_D_COUNTRY_CODE,MANUFACTURER_D_POSTAL_CODE,DEVICE_OPERATOR,EXPIRATION_DATE_OF_DEVICE,MODEL_NUMBER,CATALOG_NUMBER,LOT_NUMBER,OTHER_ID_NUMBER,DEVICE_AVAILABILITY,DATE_RETURNED_TO_MANUFACTURER,DEVICE_REPORT_PRODUCT_CODE,DEVICE_AGE_TEXT,DEVICE_EVALUATED_BY_MANUFACTUR,COMBINATION_PRODUCT_FLAG,UDI-DI,UDI-PUBLIC
0,13137061,None,2124215-2021-35010,M,Y,None,None,2022-01-01 00:00:00,Y,Y,2022-03-05 00:00:00,2021-10-06 00:00:00,None,1.0,Y,N,None,None,None,None,NaN,None,None,TIMOTHY,DEGROOT,4100 HAMLINE AVENUE NORTH,None,SAINT PAUL,MN,55112,NaN,US,55112,651.0,651.0,6515.0,None,NaN,65158261,6515826168,BOSTON SCIENTIFIC CORPORATION,"ROAD 698, LOT NO. 12",None,DORADO,PR,00646,-260.0,*,00646-2602,2021-10-06 00:00:00,Follow-Up,2019-06-27 00:00:00,Y,None,I,None,IN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"I,F",05,2022-01-01 00:00:00,2025-08-06 00:00:00,NC,US,P150012/S000,NaN,N,1.0,2022-03-07 00:00:00,2022-01-07 00:00:00,13137061,None,None,None,1608208.0,2019-01-01 00:00:00,1970-01-01 00:00:00.000002,U,2022-01-01 00:00:00,INGEVITY MRI,DRUG ELUTING PERMANENT RIGHT VENTRICULAR OR AT...,BOSTON SCIENTIFIC CORPORATION,4100 HAMLINE AVENUE NORTH,None,SAINT PAUL,MN,55112,None,US,55112,0LP,2021-06-27 00:00:00,7741,7741,1049726,None,R,2021-11-04 00:00:00,NVN,DA,Y,Y,00802526523458,00802526523458
1,13137065,None,2124215-2021-36973,M,Y,None,None,2022-01-01 00:00:00,Y,N,2022-01-01 00:00:00,2021-10-01 00:00:00,N,0.0,None,N,None,None,None,None,NaN,None,None,TIMOTHY,DEGROOT,4100 HAMLINE AVENUE NORTH,None,SAINT PAUL,MN,55112,NaN,US,55112,651.0,651.0,6515.0,None,NaN,65158261,6515826168,BOSTON SCIENTIFIC CORPORATION,"ROAD 698, LOT NO. 12",None,DORADO,PR,00646,-260.0,*,00646-2602,2021-10-06 00:00:00,Initia

## 3. Event Type Breakdown

The master table provides EVENT_TYPE (Death, Injury, Malfunction, etc.)

In [ ]:
# Get event type breakdown
breakdown = db.event_type_breakdown_for(results)

print(f"Total events: {breakdown['total']:,}")
print(f"Deaths: {breakdown['deaths']:,}")
print(f"Injuries: {breakdown['injuries']:,}")
print(f"Malfunctions: {breakdown['malfunctions']:,}")

Total events: 129,615
Deaths: 1,560
Injuries: 58,762
Malfunctions: 69,268


## 4. Yearly Trends

Analyze event counts over time.

In [ ]:
# Get yearly trends
trends = db.get_trends_by_year(device_name='pacemaker')
print(trends)

   year  event_count  deaths  injuries  no_patient_impact
0  2022        58265     685     31070                 66
1  2023        71102     833     25126                  0


## 5. Date Range Filtering

In [ ]:
# Query with date range
recent = db.query_device(
    device_name='pacemaker',
    start_date='2023-01-01',
    end_date='2023-12-31'
)

print(f"2023 events: {len(recent):,}")

2023 events: 71,234


## 6. Export to CSV

In [ ]:
results.to_csv('pacemaker_2022_2023.csv', index=False)
print("✓ Exported to pacemaker_2022_2023.csv")

✓ Exported to pacemaker_2022_2023.csv


In [ ]:
db.close()
print("\n✓ Complete! See notebook 03 for visualization.")


✓ Complete! See notebook 03 for visualization.
